In [1]:
### 전처리 함수나 클래스를 만들어서 쓰자
### 학습 데이터 전처리 할 때는 멀티 쓰레드 활용

### input -> 100개 sample point * 4 channel의 ndarray
### output -> df_data 중 

In [2]:
### import
import os
import ast

from tqdm import tqdm
def tqdm_close():
    for instance in tqdm._instances:
        instance.close()
import multiprocessing as mp
print('cpu 개수는:', mp.cpu_count())
from joblib import Parallel, delayed

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt

import librosa

cpu 개수는: 12


In [3]:
### make input file list
def get_file_list(cls):
    if cls == 1:
        folder_path = 'dataset' # siren
    else:
        folder_path = 'dataset2' # others
    file_list = os.listdir(folder_path)
    return file_list

def rsp_50to20k(original_array): # shape: (4,100)
    rsp_array_list = []
    for array in original_array:
        rsp_array_list.append(np.round(librosa.resample(array, orig_sr=50, target_sr=5000), 3))
    rsp_array = np.array(rsp_array_list) # (4, 40000)
    sum_array = np.sum(np.array(rsp_array_list), axis=0) # (40000, 0)
    rt_array = np.round(rsp_array/sum_array*100, 3) # (4, 40000)
    return rsp_array, sum_array, rt_array

def np_ast(str_list):
    return np.array(ast.literal_eval(str_list))

In [103]:
for cls in [0]:  
    
    if cls == 1:
        folder_path = 'dataset'
    else:
        folder_path = 'dataset2'
    file_list = get_file_list(cls)  

    ch1_raw_list, ch2_raw_list, ch3_raw_list, ch4_raw_list = [], [], [], []
    ch1_rsp_list, ch2_rsp_list, ch3_rsp_list, ch4_rsp_list = [], [], [], []
    ch1_rt_list, ch2_rt_list, ch3_rt_list, ch4_rt_list, mean_list = [], [], [], [], []
    x_list, y_list, i_list = [], [], []
    deg_list, sin_list, cos_list, dt_list, cls_list = [], [], [], [], []    

    for i in tqdm(range(len(file_list))):
        x = float(file_list[i].split('.')[0].split('_')[0])
        y = float(file_list[i].split('.')[0].split('_')[1])

        dt = round(np.sqrt(x**2 + y**2), 1)
        theta = np.arctan2(y, x)
        deg = round(theta * (180 / np.pi), 1)
        cos_val = round(np.cos(theta), 2)
        sin_val = round(np.sin(theta), 2)

        df_to_prcs = pd.read_csv(f'{folder_path}/{file_list[i]}')

        for j in range(0,151,25):
            df_sub = df_to_prcs.iloc[j:j+100, :]
            ch1_raw, ch2_raw, ch3_raw, ch4_raw = df_sub.iloc[:, :4].T.values.astype(float) # type: np.ndarray, len: 100
            (ch1_rsp, ch2_rsp, ch3_rsp, ch4_rsp), sum_rsp, (ch1_rt, ch2_rt, ch3_rt, ch4_rt) = rsp_50to20k(df_sub.iloc[:, :4].T.values.astype(float))

            ch1_raw_list.append(list(ch1_raw))
            ch2_raw_list.append(list(ch2_raw))
            ch3_raw_list.append(list(ch3_raw))
            ch4_raw_list.append(list(ch4_raw))

            ch1_rsp_list.append(list(ch1_rsp))
            ch2_rsp_list.append(list(ch2_rsp))
            ch3_rsp_list.append(list(ch3_rsp))
            ch4_rsp_list.append(list(ch4_rsp))

            ch1_rt_list.append(list(ch1_rt))
            ch2_rt_list.append(list(ch2_rt))
            ch3_rt_list.append(list(ch3_rt))
            ch4_rt_list.append(list(ch4_rt))

            mean_list.append(list(np.round(sum_rsp/4, 3)))

            x_list.append(x)
            y_list.append(y)
            deg_list.append(deg)
            sin_list.append(sin_val)
            cos_list.append(cos_val)
            dt_list.append(dt)
            cls_list.append(cls)
            i_list.append(j)

    df = pd.DataFrame({
        'ch1_raw': ch1_raw_list, 'ch2_raw': ch2_raw_list, 'ch3_raw': ch3_raw_list, 'ch4_raw': ch4_raw_list,
        'ch1_rsp': ch1_rsp_list, 'ch2_rsp': ch2_rsp_list, 'ch3_rsp': ch3_rsp_list, 'ch4_rsp': ch4_rsp_list,
        'ch1_rt': ch1_rt_list, 'ch2_rt': ch2_rt_list, 'ch3_rt': ch3_rt_list, 'ch4_rt': ch4_rt_list,
        'mean': mean_list, 'x': x_list, 'y': y_list, 'i': i_list,
        'deg': deg_list, 'sin': sin_list, 'cos': cos_list, 'dt': dt_list, 'cls': cls_list
    })
    
tqdm_close()

 63%|██████████████████████████████████████████████████▎                             | 196/312 [07:41<04:32,  2.35s/it]

KeyboardInterrupt



In [50]:
df1.to_csv('siren_class.csv', index=False)

In [44]:
df0.to_csv('other_class.csv', index=False)

In [45]:
df0 = pd.read_csv('other_class.csv')

In [47]:
df1 = pd.read_csv('siren_class.csv')

In [48]:
for i in [1, 2, 3, 4]:
    fre_array_list = []
    col_name = f'ch{i}_rt'
    rsp_series = np.array(df1[col_name])
    for j in tqdm(range(len(rsp_series))):
        rsp_array = np_ast(rsp_series[j])
        mel_spectrogram = librosa.feature.melspectrogram(y=rsp_array, sr=5000, n_mels=64, hop_length=101, win_length=2048)
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        mfcc = librosa.feature.mfcc(S=log_mel_spectrogram, sr=5000, n_mfcc= 36)
        fre_array = np.round(np.vstack((log_mel_spectrogram, mfcc)), 3)
        fre_array_list.append(fre_array.tolist())
    df1[f'ch{i}_fre'] = fre_array_list
tqdm_close()

100%|██████████████████████████████████████████████████████████████████████████████| 3101/3101 [04:37<00:00, 11.17it/s]


In [42]:
for i in [1, 2, 3, 4]:
    fre_array_list = []
    col_name = f'ch{i}_fre'
    rsp_series = ddf[col_name]
    for fre_array in tqdm(rsp_series):
        fre_array = np.round(np.array(fre_array), 3).tolist()
        fre_array_list.append(fre_array)
    df0[f'ch{i}_fre'] = fre_array_list
tqdm_close()

100%|████████████████████████████████████████████████████████████████████████████| 2184/2184 [00:01<00:00, 1106.85it/s]


In [49]:
df1

,ch1_raw,ch2_raw,ch3_raw,ch4_raw,ch1_rsp,ch2_rsp,ch3_rsp,ch4_rsp,ch1_rt,ch2_rt,ch3_rt,ch4_rt,mean,x,y,i,deg,sin,cos,dt,cls,ch1_fre,ch2_fre,ch3_fre,ch4_fre
0,"[85.0, 68.0, 86.0, 61.0, 66.0, 92.0, 96.0, 81....","[97.0, 70.0, 103.0, 101.0, 67.0, 117.0, 120.0,...","[143.0, 93.0, 89.0, 82.0, 64.0, 99.0, 116.0, 1...","[76.0, 77.0, 93.0, 69.0, 67.0, 65.0, 99.0, 67....","[79.173, 82.597, 85.535, 87.988, 89.971, 91.51...","[91.761, 95.742, 99.162, 102.022, 104.339, 106...","[134.869, 140.718, 145.74, 149.939, 153.339, 1...","[69.815, 72.877, 75.517, 77.737, 79.55, 80.979...","[21.078, 21.074, 21.07, 21.066, 21.061, 21.055...","[24.429, 24.428, 24.427, 24.426, 24.424, 24.42...","[35.906, 35.903, 35.901, 35.898, 35.894, 35.89...","[18.587, 18.594, 18.602, 18.611, 18.621, 18.63...","[93.904, 97.984, 101.488, 104.421, 106.8, 108....",-100.0,10.0,0,174.3,0.1,-1.00,100.5,1,"[[-4.191, -3.798, -3.703, -3.864, -4.232, -4.7...","[[-0.736, -0.238, -0.007, 0.0, -0.166, -0.445,...","[[-0.153, 0.0, -0.159, -0.587, -1.221, -1.959,...","[[-1.382, -0.978, -0.908, -1.133, -1.595, -2.1..."
1,"[98.0, 87.0, 82.0, 60.0, 57.0, 85.0, 78.0, 85....","[116.0, 96.0, 67.0, 99.0, 118.0, 77.0, 84.0, 7...","[80.0, 94.0, 88.0, 105.0, 99.0, 74.0, 73.0, 80...","[77.0, 46.0, 54.0, 57.0, 69.0, 79.0, 81.0, 68....","[93.341, 97.452, 101.003, 103.996, 106.446, 10...","[110.34, 115.2, 119.398, 122.934, 125.831, 128...","[76.462, 79.914, 82.922, 85.488, 87.624, 89.35...","[72.213, 75.316, 77.971, 80.18, 81.957, 83.324...","[26.491, 26.49, 26.49, 26.489, 26.488, 26.488,...","[31.315, 31.314, 31.314, 31.313, 31.312, 31.31...","[21.7, 21.723, 21.748, 21.775, 21.805, 21.837,...","[20.494, 20.473, 20.449, 20.423, 20.395, 20.36...","[88.089, 91.97, 95.324, 98.15, 100.464, 102.29...",-100.0,10.0,25,174.3,0.1,-1.00,100.5,1,"[[-2.387, -2.08, -2.064, -2.279, -2.644, -3.06...","[[-0.412, -0.049, 0.0, -0.246, -0.757, -1.475,...","[[-0.488, -0.086, 0.0, -0.183, -0.568, -1.069,...","[[-6.427, -5.906, -5.608, -5.491, -5.517, -5.6..."
2,"[66.0, 77.0, 93.0, 67.0, 46.0, 86.0, 64.0, 81....","[75.0, 93.0, 82.0, 90.0, 64.0, 69.0, 59.0, 58....","[65.0, 87.0, 84.0, 66.0, 77.0, 81.0, 58.0, 69....","[72.0, 100.0, 77.0, 73.0, 56.0, 61.0, 48.0, 70...","[61.745, 64.518, 66.928, 68.979, 70.68, 72.051...","[72.341, 75.631, 78.506, 80.965, 83.023, 84.7,...","[62.989, 65.85, 68.347, 70.483, 72.268, 73.721...","[69.899, 73.113, 75.93, 78.352, 80.392, 82.071...","[23.128, 23.115, 23.102, 23.087, 23.071, 23.05...","[27.097, 27.097, 27.098, 27.099, 27.1, 27.1, 2...","[23.594, 23.593, 23.591, 23.59, 23.589, 23.587...","[26.182, 26.195, 26.209, 26.224, 26.241, 26.25...","[66.744, 69.778, 72.428, 74.695, 76.591, 78.13...",-100.0,10.0,50,174.3,0.1,-1.00,100.5,1,"[[-4.261, -3.768, -3.534, -3.526, -3.697, -3.9...","[[-0.359, -0.018, 0.0, -0.261, -0.742, -1.356,...","[[-0.778, -0.31, -0.122, -0.175, -0.43, -0.833...","[[-2.044, -1.828, -1.929, -2.302, -2.876, -3.5..."
3,"[79.0, 125.0, 153.0, 130.0, 94.0, 83.0, 85.0, ...","[89.0, 134.0, 120.0, 133.0, 56.0, 62.0, 52.0, ...","[88.0, 142.0, 110.0, 139.0, 59.0, 63.0, 59.0, ...","[61.0, 93.0, 132.0, 90.0, 88.0, 62.0, 72.0, 11...","[72.564, 75.904, 78.834, 81.356, 83.482, 85.23...","[88.88, 92.998, 96.619, 99.743, 102.386, 104.5...","[88.96, 93.136, 96.823, 100.022, 102.749, 105....","[53.658, 56.085, 58.202, 60.009, 61.517, 62.74...","[23.865, 23.86, 23.855, 23.849, 23.843, 23.837...","[29.231, 29.233, 29.236, 29.239, 29.242, 29.24...","[29.257, 29.277, 29.298, 29.321, 29.346, 29.37...","[17.647, 17.63, 17.611, 17.591, 17.57, 17.546,...","[76.016, 79.531, 82.62, 85.282, 87.533, 89.395...",-100.0,10.0,75,174.3,0.1,-1.00,100.5,1,"[[-2.544, -2.036, -1.807, -1.824, -2.039, -2.3...","[[-2.613, -2.41, -2.497, -2.799, -3.21, -3.602...","[[-0.193, 0.0, -0.115, -0.474, -0.978, -1.487,...","[[-3.981, -3.315, -2.89, -2.66, -2.589, -2.629..."
4,"[52.0, 93.0, 61.0, 95.0, 111.0, 73.0, 56.0, 45...","[79.0, 48.0, 32.0, 61.0, 62.0, 66.0, 50.0, 37....","[65.0, 58.0, 44.0, 68.0, 81.0, 78.0, 61